In [ ]:
!pip install redis


In [6]:
import redis
import json

In [7]:
# Criar conexão com o REDIS
r = redis.Redis(
  host='redis-12148.c308.sa-east-1-1.ec2.cloud.redislabs.com',
  port=12148,
  password='EgQ7nsyeQsa139Yi5L5r89lRu72CJb9z')

In [ ]:
# Agora você pode realizar operações como de costume
r.set('nome', 'Shandler Menezes Gama')
valor = r.get('nome')
print(valor.decode('utf-8'))  # Convertendo bytes para string

In [ ]:
# Dados de exemplo
period = "2022_Maio"
comment = "Some comment - 2"
incomes = {'Salary': 1600, 'Blog': 60, 'Other Income': 60}
expenses = {'Rent': 6600, 'Utilities': 2600, 'Groceries': 3600, 'Car': 1600, 'Other Expenses': 560, 'Saving': 160}
# Converter os dicionários de incomes e expenses em strings JSON
incomes_json = json.dumps(incomes)
expenses_json = json.dumps(expenses)

In [ ]:
# Gravar dados
r.hset(period, "incomes", incomes_json)
r.hset(period, "expenses", expenses_json)
r.hset(period, "comment", comment)

In [8]:
# Recuperar os dados do período do Redis
comment = r.hget(period, "comment")
incomes_json = r.hget(period, "incomes")
expenses_json = r.hget(period, "expenses")

In [9]:
# Verificar se os dados foram encontrados
if comment is not None and incomes_json is not None and expenses_json is not None:
    # Converter os valores de JSON de volta para dicionários Python
    comment = comment.decode('utf-8')
    incomes = json.loads(incomes_json.decode('utf-8'))
    expenses = json.loads(expenses_json.decode('utf-8'))

    # Exibir os dados recuperados
    print("Comment:", comment)
    print("Incomes:", incomes)
    print("Expenses:", expenses)
else:
    print("Nenhum dado encontrado para o período", period)

Comment: janeiro
Incomes: {'Salary': 10, 'Blog': 10, 'Other Income': 10}
Expenses: {'Rent': 10, 'Utilities': 10, 'Groceries': 10, 'Car': 10, 'Other Expenses': 10, 'Saving': 10}


In [15]:
# Funções para pegar um periodo especifico

# Gerar o retorno esperado
def generate_return(period, incomes, expenses, comment):
    """Gera o retorno no formato desejado"""
    return [{
        "key": period,
        "incomes": incomes,
        "expenses": expenses,
        "comment": comment
    }]

# Busca os valores no banco REDIS
def get_period(period):
    # Recuperar os dados do período do Redis
    comment = r.hget(period, "comment")
    incomes_json = r.hget(period, "incomes")
    expenses_json = r.hget(period, "expenses")
    # Verificar se os dados foram encontrados
    if comment is not None and incomes_json is not None and expenses_json is not None:
        # Converter os valores de JSON de volta para dicionários Python
        comment = comment.decode('utf-8')
        incomes = json.loads(incomes_json.decode('utf-8'))
        expenses = json.loads(expenses_json.decode('utf-8'))
    else:
        print("Nenhum dado encontrado para o período", period)
    # Gerar o retorno no formato desejado
    result = generate_return(period, incomes, expenses, comment)
    # Converter o retorno em uma string JSON
    result_json = json.dumps(result)
    return result_json

In [ ]:
# Retorna todos os periodos do Bando REDIS
def fetch_all_periods():
    """Recupera todos os períodos do Redis"""
    periods = {}
    keys = r.keys('*')
    for key in keys:
        data = r.hgetall(key)
        period_data = {}
        for field, value in data.items():
            field = field.decode('utf-8')
            if field == 'incomes' or field == 'expenses':
                period_data[field] = json.loads(value.decode('utf-8'))
            else:
                period_data[field] = value.decode('utf-8')
        periods[key.decode('utf-8')] = period_data
    return periods

# Exemplo de uso
all_periods = fetch_all_periods()
for period, data in all_periods.items():
    print("Period:", period)
    print("Data:", data)

""" Exemplo do retorno

Period: 2024_April
Data: {'expenses': {'Rent': 600, 'Utilities': 200, 'Groceries': 300, 'Car': 100, 'Other Expenses': 50, 'Saving': 10}, 'comment': 'Some comment 3', 'incomes': {'Salary': 1500, 'Blog': 50, 'Other Income': 10}}

"""

In [ ]:
def fetch_all_periods():
    """Recupera todos os períodos do Redis"""
    all_periods = []
    keys = r.keys('*')
    for key in keys:
        data = r.hgetall(key)
        period_data = {}
        period_data['key'] = key.decode('utf-8')  # Adicionando a chave 'key' ao dicionário
        for field, value in data.items():
            field = field.decode('utf-8')
            if field == 'incomes' or field == 'expenses':
                period_data[field] = json.loads(value.decode('utf-8'))
            else:
                period_data[field] = value.decode('utf-8')

        all_periods.append(period_data)
    return all_periods

# Exemplo de uso
all_periods = fetch_all_periods()
print(json.dumps(all_periods, indent=2))

"""Exeplo do Retorno
[
  {
    "key": "2024_April",
    "expenses": {
      "Rent": 600,
      "Utilities": 200,
      "Groceries": 300,
      "Car": 100,
      "Other Expenses": 50,
      "Saving": 10
    },
    .
    .
    .
]
"""

In [16]:
import json

#period = "2022_April"
#period = "2022_Maio"
#period = "2024_April"
period = "2024_January"

# Suponha que data seja uma string que representa um dicionário JSON
data = '[{"key": "2022_Maio", "incomes": {"Salary": 1600, "Blog": 60, "Other Income": 60}, "expenses": {"Rent": 6600, "Utilities": 2600, "Groceries": 3600, "Car": 1600, "Other Expenses": 560, "Saving": 160}, "comment": "Some comment - 2"}]'

# Converta a string JSON em um dicionário Python
parsed_data = json.loads(get_period(period))

# Agora parsed_data é uma lista de dicionários
for period_data in parsed_data:
    key = period_data["key"]
    incomes = period_data["incomes"]
    expenses = period_data["expenses"]
    comment = period_data["comment"]

    print("Key:", key)
    print("Incomes:", incomes)
    print("Expenses:", expenses)
    print("Comment:", comment)

Key: 2024_January
Incomes: {'Salary': 10, 'Blog': 10, 'Other Income': 10}
Expenses: {'Rent': 10, 'Utilities': 10, 'Groceries': 10, 'Car': 10, 'Other Expenses': 10, 'Saving': 10}
Comment: janeiro
